In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
from shuler_lab_to_nwb.behavior_datainterface import ShulerBehaviorInterface
from datetime import datetime
from dateutil import tz

In [8]:
trials_file = "/media/luiz/storage2/taufferconsulting/client_catalystneuro/project_shuler/ES029_2022-09-14_bot72_0_g0/data_2022-09-14_12-13-31.csv"
reference_ts_file = "/media/luiz/storage2/taufferconsulting/client_catalystneuro/project_shuler/ES029_2022-09-14_bot72_0_g0/ES029_2022-09-14_bot72_0_g0_tcat.imec0.ap.SY_384_6_0.txt"

data_interface = ShulerBehaviorInterface(
    trials_file_path=trials_file,
    reference_timestamps_file_path=reference_ts_file
)

In [9]:
metadata = data_interface.get_metadata()
session_start_time = datetime(2020, 1, 1, 12, 30, 0, tzinfo=tz.gettz("US/Pacific"))
metadata['NWBFile'].update(session_start_time=session_start_time)
metadata

DeepDict: {'NWBFile': {'session_description': 'Auto-generated by neuroconv', 'identifier': '07fcdc80-aec8-43a8-ac34-9185915dc41e', 'session_start_time': datetime.datetime(2020, 1, 1, 12, 30, tzinfo=tzfile('/usr/share/zoneinfo/US/Pacific'))}}

In [10]:
data_interface.run_conversion(
    nwbfile_path="quicktest.nwb",
    metadata=metadata,
    overwrite=True
)

/media/luiz/storage2/Github/shuler-lab-to-nwb/shuler_lab_to_nwb/behavior_datainterface/datainterface.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.df["session_time_sync"] = aligned_timestamps


NWB file saved at quicktest.nwb!


In [11]:
from pynwb import NWBHDF5IO
from nwbwidgets import nwb2widget

io = NWBHDF5IO('quicktest.nwb', mode='r')
nwbfile = io.read()

nwb2widget(nwbfile)